In [ ]:
!pip install datasets transformers jsonlines

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from huggingface_hub import interpreter_login

interpreter_login()

/users/klee12/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved to /users/klee12/.cache/huggingface/token
Login successful


In [2]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset("andstor/smart_contract_code_comments", 'default')

# Print basic information about the dataset
print(dataset)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Generating train split: 1267441 examples [00:44, 28621.72 examples/s]
Generating validation split: 130849 examples [00:04, 26275.12 examples/s]
Generating test split: 143080 examples [00:04, 29798.38 examples/s]

DatasetDict({
    train: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 1267441
    })
    validation: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 130849
    })
    test: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_docume

# Training Dataset

In [3]:
train = dataset['train']

## Split Full Code and Code Comment

In [4]:
remove_cols = ['file_path', 'contract_address', 'language', 'class_name', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']

In [5]:
code = train.remove_columns(remove_cols)

In [6]:
# drop duplicates
full_code = pd.DataFrame(code)
full_code.drop_duplicates(inplace=True)

## Function Headers and Function Comments

In [7]:
remove_func_col = ['file_path', 'contract_address', 'class_code', 'class_documentation', 'language', 'class_name', 'class_documentation_type', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']

In [8]:
func_code = train.remove_columns(remove_func_col)

In [9]:
func = pd.DataFrame(func_code)

In [10]:
import re

# Define the function that extracts function headers
def extract_function_header(func_code):
    # Regular expression to match Solidity function declarations
    func_header_regex = r'function\s*[\w\d_]*\s*\([^)]*\)\s*[^{]*'
    # Find all matches in the func_code
    matches = re.findall(func_header_regex, func_code)
    # Assume each func_code contains one function and take the first match
    # If there are multiple matches, this will need adjustment
    return matches[0] if matches else None

# Assume 'data' is your DataFrame loaded from CSV and it has a column 'func_code'
# You can apply the function directly on the DataFrame column
func['func_header'] = func['func_code'].apply(extract_function_header)

# filter out rows with no function headers
func = func[func['func_header'].apply(lambda x: x is not None and len(x) > 0)]

In [11]:
grouped_data = func.groupby('contract_name').agg({
    'func_header': list,
    'func_documentation': list
}).reset_index()

In [12]:
print(grouped_data)

      contract_name                                        func_header  \
0        $$$$$Token  [function totalSupply() public constant return...   
1               $33  [function balanceOf(address account) external ...   
2             $4RFI  [function totalSupply() external view returns ...   
3              $AVG  [function totalSupply() public constant return...   
4          $BANKINU  [function name() external view returns (string...   
...             ...                                                ...   
60688   zombittoken  [function totalSupply() constant returns (uint...   
60689     zp_minter  [function supportsInterface(bytes4 interfaceId...   
60690  zrankadictos  [function TokenERC20(\r\n    uint256 initialSu...   
60691       zsToken  [function deposit(uint256 amount, uint256 _tok...   
60692     zukiducks  [function tokenURI(uint256 _tokenId)\r\n    pu...   

                                      func_documentation  
0      [// -----------------------------------------

## Join On Contract Name

In [13]:
cleaned_data = pd.merge(grouped_data, full_code, on='contract_name', how='left')
cleaned_data = cleaned_data.drop_duplicates(subset=['contract_name'], keep='first')


In [14]:
print(cleaned_data.iloc[0])

contract_name                                                 $$$$$Token
func_header            [function totalSupply() public constant return...
func_documentation     [// ------------------------------------------...
class_code             contract $$$$$Token is ERC20Interface, SafeMat...
class_documentation    /**\r\n ERC20 Token, with the addition of symb...
Name: 0, dtype: object


# Generate Prompts into JSON File

Format:
- Task Description: `class_documentation` - contract comment
- Key Functions: 
  - `func_header, func_documentation`: function header and comment
- Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).

Example:

- Task Description: Create a token according to the ERC-20 standard with a mechanism for freezing assets for a specified period.
- Key Functions: 
  - `mint(address recipient, uint256 amount)`: Mints tokens to a specified address.
  - `freeze(address target, uint256 untilTimestamp)`: Freezes the assets of a specified address until a certain timestamp.
  - `transfer(address recipient, uint256 amount)`: Allows token transfers, checking if the sender's assets are frozen.
- Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).

In [15]:
# get 100 examples to use (TEST WITH 10)
raw = cleaned_data[:100]

In [35]:
def write_user_prompt(class_documentation, func_code_list, func_comment_list):
    prompt = 'Generate a smart contract using Solidity 0.4.17 that accomplishes the following requirements:\n'
    
    prompt += 'Task Description: {}\n'.format(class_documentation.replace("/", "").replace("*", ""))
    prompt += 'Key Functions:\n'
    
    for idx in range(len(func_code_list)):
        comment =  func_comment_list[idx].replace("/", "").replace("-", "").replace("\n", "").replace("@notice", "").replace("@param", "Parameter: ").replace("*", "")
        prompt += '{}: {}\n'.format(func_code_list[idx], comment)
    
    prompt += 'Constraints: The contract must be gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).\n'
    
    return prompt

In [36]:
output = []

for idx, row in raw.iterrows():
    prompt = [{"role": "system", "content": "DEFI is an experienced Ethereum developer that writes properly functioning Solidity contracts."}]
    prompt.append({"role":"user","content":"{}".format(write_user_prompt(row['class_documentation'], row['func_header'], row['func_documentation']))})
    prompt.append({"role": "assistant", "content": "{}".format(row['class_code'])})
        
    output.append({"messages":prompt})

In [30]:
with open('test-prompt.txt', 'w') as writefile:
    writefile.write(output[0]['messages'][1]['content'])

In [27]:
import json

with open('3.json', 'w') as f:
    json.dump(output[3], f)


In [20]:
import jsonlines

with jsonlines.open('prompts_final.jsonl', mode='w') as writer:
    writer.write_all(output)